#Protein Secondary Structure Prediction
Using a Hidden Markov Model to Classify Amino Acid Sequences into Secondary Structure states in order to understand protein folding
#Names
Anila Chundi, Ranjini Ramesh, Shalini Sivakumar



In [1]:
#Install libraries

! pip install nose
! pip install numpy
! pip install pandas
! pip install matplotlib


#CB513 Dataset
- The CB513 dataset is a benchmark dataset widely used for training and testing protein secondary structure prediction models. It contains 513 protein sequences with annotated secondary structures.
- dataset is from here: https://huggingface.co/datasets/proteinea/secondary_structure_prediction/blob/main/CB513.csv

#DSSP Dataset
- The DSSP (Dictionary of Secondary Structure in Proteins) dataset provides detailed secondary structure assignments for protein entries in the Protein Data Bank (PDB).


In [8]:
#Download data here
import requests

url = 'https://huggingface.co/datasets/proteinea/secondary_structure_prediction/resolve/main/CB513.csv'
output_file = 'CB513.csv'

response = requests.get(url)
if response.status_code == 200:
    with open(output_file, 'wb') as file:
        file.write(response.content)
    print(f"File downloaded successfully as {output_file}")
else:
    print(f"Failed to download file. HTTP Status Code: {response.status_code}")

File downloaded successfully as CB513.csv


In [9]:
import copy
import sys

import nose.tools as nt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#Data Parsing
- read and understand data in datasets

**CB513_data**
- "input" column contains the amino acid sequences for each protein
  - each row represents a protein sequence as a long string of one-letter amino acid codes
- "dssp3" column represents the secondary structure annotation for the corresponding protein in "3-class DSSP" format
  - H: Helix, E: Strand(beta sheet), C: Coil
  - each char corresponds to the secondary structure of the amino acid in the input column at the same position
- "dssp8" column represents the secondary structure annotation for the corresponding protein in "8-class DSSP" format
  - H: Alpha Helix, E: Extended Strand, C: Coil, G: 3/10 helix, I: π-helix, B: Isolated β-bridge, T: Turn, S: Bend
  - each char corresponds to the secondary structure of the amino acid in the input column at the same position
- "disorder" column indicates a region without a stable secondary structure
  - values: 1 = Disordered, 0 = Ordered
- "cb513_mask" column serves as a mask used to exclude padding, invalid data, etc
  - 1 = valid, 0 = invalid

In [10]:
# Load the CSV file into a DataFrame
cb513_data = pd.read_csv('CB513.csv')

print(cb513_data.head())


                                               input  \
0  RTDCYGNVNRIDTTGASCKTAKPEGLSYCGVSASKKIAERDLQAMD...   
1  GKITFYEDRGFQGRHYECSSDHSNLQPYFSRCNSIRVDSGCWMLYE...   
2  MFKVYGYDSNIHKCVYCDNAKRLLTVKKQPFEFINIMPEKGVFDDE...   
3  APAFSVSPASGASDGQSVSVSVAAAGETYYIAQCAPVGGQDACNPA...   
4  TPAFNKPKVELHVHLDGAIKPETILYFGKKRGIALPADTVEELRNI...   

                                               dssp3  \
0  CCCCCCCHHHCCCCCECHHHHCCCCCCCCEHHHHHHHHHHCHHHHH...   
1  CEEEEEEECCCEEEEEEECCCECCCCCCCCCCCEEEEEECEEEEEC...   
2  CEEEEECCCCCCCCHHHHHHHHHHHHCCCCEEEEECCCECCECCHH...   
3  CCEEEEECCCCCCCCCEEEEEEECCCCEEEEEEECEECCEECCCCC...   
4  CCCCCCCEEEEEEEHHHCCCHHHHHHHHHHHCCCCCCCCHHHHHHH...   

                                               dssp8  \
0  CCCTTCCGGGSCCCCBCHHHHTTTTCSCCBHHHHHHHHHHTHHHHH...   
1  CEEEEEEETTTEEEEEEECSCBSCCTTTCSCCSEEEEEESEEEEES...   
2  CEEEEECCTTTSCCHHHHHHHHHHHHTTCCEEEEESCSBTTBCCHH...   
3  CCEEEEECCSSCCSSCEEEEEEESCCSEEEEEEECEETTEECCCTT...   
4  CCSCCSCEEEEEEEGGGSCCHHHHHHHHHHHTCCCSCSSHHHH

#HMM Training
- Use previous dataset to train the HMM model
  - Use input column as the 'observed states'
  - Use dssp3 as hidden states

In [ ]:
observed_states = cb513_data['input'].tolist()
hidden_states = cb513_data['dssp3'].tolist()

print(observed_states)
print(hidden_states)

for obs, hid in zip(observed_states, hidden_states):
  assert(len(obs) == len(hid))

hmm_data = list(zip(observed_states, hidden_states))
print('--------------------------------')
print(hmm_data)



['RTDCYGNVNRIDTTGASCKTAKPEGLSYCGVSASKKIAERDLQAMDRYKTIIKKVGEKLCVEPAVIAGIISRESHAGKVLKNGWGDRGNGFGLMQVDKRSHKPQGTWNGEVHITQGTTILINFIKTIQKKFPSWTKDQQLKGGISAYNAGAGNVRSYARMDIGTTHDDYANDVVARAQYYKQHGY', 'GKITFYEDRGFQGRHYECSSDHSNLQPYFSRCNSIRVDSGCWMLYEQPNFQGPQYFLRRGDYPDYQQWMGLNDSIRSCRLIPHTGSHRLRIYEREDYRGQMVEITEDCSSLHDRFHFSEIHSFNVLEGWWVLYEMTNYRGRQYLLRPGDYRRYHDWGATNARVGSLRRAVDFY', 'MFKVYGYDSNIHKCVYCDNAKRLLTVKKQPFEFINIMPEKGVFDDEKIAELLTKLGRDTQIGLTMPQVFAPDGSHIGGFDQLREYFK', 'APAFSVSPASGASDGQSVSVSVAAAGETYYIAQCAPVGGQDACNPATATSFTTDASGAASFSFTVRKSYAGQTPSGTPVGSVDCATDACNLGAGNSGLNLGHVALTFG', 'TPAFNKPKVELHVHLDGAIKPETILYFGKKRGIALPADTVEELRNIIGMDKPLSLPGFLAKFDYYMPVIAGCREAIKRIAYEFVEMKAKEGVVYVEVRYSPHLLANSKVDPMPWNQTEGDVTPDDVVDLVNQGLQEGEQAFGIKVRSILCCMRHQPSWSLEVLELCKKYNQKTVVAMDLAGDETIEGSSLFPGHVEAYEGAVKNGIHRTVHAGEVGSPEVVREAVDILKTERVGHGYHTIEDEALYNRLLKENMHFEVCPWSSYLTGAWDPKTTHAVVRFKNDKANYSLNTDDPLIFKSTLDTDYQMTKKDMGFTEEEFKRLNINAAKSSFLPEEEKKELLERLYREYQ', 'GNNVVVLGTQWGDEGKGKIVDLLTERAKYVVRYQGGHNAGHTLVINGEKTVLHLIPSGILRENVTSIIGNGVVLSP

In [ ]:
# #Baum-Welch Implementation
# def initialize_hmm(states, observations):
#     # Initialize random probabilities for A, E, and π
#     n_states = len(states)
#     n_obs = len(observations)
    
#     A = np.random.dirichlet(np.ones(n_states), size=n_states)  # Transition probabilities
#     E = np.random.dirichlet(np.ones(n_obs), size=n_states)    # Emission probabilities
#     I = np.random.dirichlet(np.ones(n_states))               # Initial probabilities
#     return A, E, I

# def forward(obs_seq, states, A, E, I):
#     n_states = len(states)
#     T = len(obs_seq)
#     alpha = np.zeros((T, n_states))
    
#     # Initialization
#     alpha[0, :] = I * E[:, obs_seq[0]]
    
#     # Induction
#     for t in range(1, T):
#         for j in range(n_states):
#             alpha[t, j] = np.sum(alpha[t-1, :] * A[:, j]) * E[j, obs_seq[t]]
    
#     return alpha

# def backward(obs_seq, states, A, E):
#     n_states = len(states)
#     T = len(obs_seq)
#     beta = np.zeros((T, n_states))
    
#     # Initialization
#     beta[T-1, :] = 1
    
#     # Induction
#     for t in range(T-2, -1, -1):
#         for i in range(n_states):
#             beta[t, i] = np.sum(A[i, :] * E[:, obs_seq[t+1]] * beta[t+1, :])
    
#     return beta

# def baum_welch(data, states, observations):
#     max_iter = 100
#     tol = 1e-4
#     n_states = len(states)
#     n_obs = len(observations)
#     A, E, I = initialize_hmm(states, observations)
    
#     for iteration in range(max_iter):
#         A_new = np.zeros_like(A)
#         E_new = np.zeros_like(E)
#         pi_new = np.zeros_like(pi)
        
#         log_likelihood = 0
        
#         for obs_seq in data:
#             T = len(obs_seq)
#             alpha = forward(obs_seq, states, A, E, pi)
#             beta = backward(obs_seq, states, A, E)
            
#             # Compute gamma and xi
#             gamma = (alpha * beta) / np.sum(alpha[-1, :])  # State probabilities
#             xi = np.zeros((T-1, n_states, n_states))       # Transition probabilities
            
#             for t in range(T-1):
#                 denom = np.sum(alpha[t, :] * A * E[:, obs_seq[t+1]] * beta[t+1, :])
#                 for i in range(n_states):
#                     xi[t, i, :] = alpha[t, i] * A[i, :] * E[:, obs_seq[t+1]] * beta[t+1, :] / denom
            
#             # Update initial probabilities
#             pi_new += gamma[0, :]
            
#             # Update transition probabilities
#             for t in range(T-1):
#                 A_new += xi[t, :, :]
            
#             # Update emission probabilities
#             for t in range(T):
#                 E_new[:, obs_seq[t]] += gamma[t, :]
            
#             # Accumulate log likelihood
#             log_likelihood += np.log(np.sum(alpha[-1, :]))
        
#         # Normalize to get probabilities
#         pi = pi_new / np.sum(pi_new)
#         A = A_new / np.sum(A_new, axis=1, keepdims=True)
#         E = E_new / np.sum(E_new, axis=1, keepdims=True)
        
#         print(f"Iteration {iteration}, Log Likelihood: {log_likelihood}")
        
#         # Check for convergence
#         if iteration > 0 and abs(log_likelihood - prev_log_likelihood) < tol:
#             break
        
#         prev_log_likelihood = log_likelihood
    
#     return A, E, I

In [ ]:
# # Create a mapping for observations
# states = ['H', 'E', 'C']
# unique_observations = set(''.join(observed_states))  # Get unique characters
# obs_mapping = {char: idx for idx, char in enumerate(unique_observations)}

# # Convert observed states to integer sequences
# integer_observed_states = [
#     [obs_mapping[char] for char in seq]
#     for seq in observed_states
# ]

# # Update hmm_data with integer sequences
# hmm_data = list(zip(integer_observed_states, hidden_states))



# A, E, I = baum_welch(hmm_data, states, list(range(len(obs_mapping))))  # Observations are now indexed

# print("pi shape:", I.shape)          # Should be (3,)
# print("E shape:", E.shape)            # Should be (3, number_of_observations)
# print("obs_seq:", observed_states)            # Should be a list of integers
# print("obs_seq[0]:", observed_states[0])      # Should be an integer between 0 and number_of_observations - 1
# print("E[:, obs_seq[0]] shape:", E[:, observed_states[0]].shape)  # Should be (3,)


UnboundLocalError: local variable 'pi' referenced before assignment

#Compute Initial, Emission, Transition log Probabilities

In [29]:
from collections import Counter

def compute_initial_probs(data):
    # Count occurrences of initial hidden states
    initial_counts = Counter()
    for _, hid_seq in data:
        if hid_seq:  # Ensure the sequence is not empty
            initial_counts[hid_seq[0]] += 1

    # Compute initial probabilities with smoothing
    total_initials = sum(initial_counts.values())
    num_hidden_states = len(initial_counts)  # Number of unique hidden states
    initial_probs = {hid: (count + 10) / (total_initials + 10 * num_hidden_states) 
                     for hid, count in initial_counts.items()}
    
    # Ensure all possible hidden states are included, even those not observed
    for state in ['H', 'E', 'C']:  # The known hidden states
        if state not in initial_probs:
            initial_probs[state] = 1e-3
    
    return initial_probs

initial_probs = compute_initial_probs(hmm_data)

def compute_emission_probs(data):
    # Count occurrences of (hidden_state, observed_state) pairs
    emission_counts = {}
    hidden_counts = Counter()

    for obs_seq, hid_seq in data:
        for obs, hid in zip(obs_seq, hid_seq):
            if hid not in emission_counts:
                emission_counts[hid] = Counter()
            emission_counts[hid][obs] += 1
            hidden_counts[hid] += 1

    # Compute emission probabilities with smoothing
    emission_probs = {
        hid: {obs: (count + 10) / (hidden_counts[hid] + 10) for obs, count in obs_count.items()}
        for hid, obs_count in emission_counts.items()
    }
    return emission_probs

emission_probs = compute_emission_probs(hmm_data)


def compute_transition_probs(data):
    # Count occurrences of (prev_hidden_state, current_hidden_state) pairs
    transition_counts = Counter()
    total_transitions = Counter()

    for _, hid_seq in data:
        for i in range(1, len(hid_seq)):
            transition_counts[(hid_seq[i-1], hid_seq[i])] += 1
            total_transitions[hid_seq[i-1]] += 1

    # Compute transition probabilities with smoothing
    transition_probs = {
        (prev, curr): (count + 10) / (total_transitions[prev] + 10)
        for (prev, curr), count in transition_counts.items()
    }
    return transition_probs

transition_probs = compute_transition_probs(hmm_data)

print(initial_probs)
print(emission_probs)
print(transition_probs)



{'C': 1.0, 'H': 0.001, 'E': 0.001}
{'C': {21: 0.040812121596348304, 1: 0.06474466668779916, 7: 0.07880305575807525, 2: 0.013931594129394237, 17: 0.028116778140552193, 11: 0.12652550163248485, 10: 0.06423748692427173, 6: 0.03369575553935398, 18: 0.0732874758297144, 0: 0.057501505689922974, 13: 0.07761435318730783, 15: 0.05258820173075094, 8: 0.05721621707293879, 19: 0.04604241290772498, 3: 0.017957333502393254, 20: 0.03185722889656703, 4: 0.024772561574793166, 12: 0.011363996576536596, 22: 0.031033061780834946, 16: 0.07011760230766792, 5: 0.000903413953783244, 9: 0.00017434304371255588, 14: 0.00019019241132278822}, 'H': {19: 0.0608372776236331, 10: 0.03611065774440999, 21: 0.05638975028562102, 2: 0.011037212338828139, 0: 0.07071160437408193, 1: 0.0421903052064632, 16: 0.12432675044883304, 18: 0.048739187204178226, 6: 0.05612453076546434, 15: 0.08735922963930146, 8: 0.10890321527664436, 20: 0.0482699526685164, 3: 0.027725640607148685, 7: 0.05367635057940264, 11: 0.03739595234209238, 13: 

#Implement Viterbi Algorithm

In [30]:

def viterbi(obs_seq, states, initial_probs, emission_probs, transition_probs):
    n = len(obs_seq)
    v = {state: [float('-inf')] * n for state in states}
    backpointer = {state: [None] * n for state in states}

    # Initialization
    for state in states:
        print('state: ', state)
        v[state][0] = np.log(initial_probs[state]) + np.log(emission_probs[state].get(obs_seq[0], 1e-3))
        # print('true initial probability: ', np.log(initial_probs[state]) + np.log(emission_probs[state].get(obs_seq[0], 1e-3)))
        # print(f"v[{state}][0]: {v[state][0]}")
        

    # Recursion
    for t in range(1, n):
        for curr_state in states:
            max_prob, prev_state = max(
                (v[prev_state][t-1] + np.log(transition_probs.get((prev_state, curr_state), 1e-6)), prev_state)
                for prev_state in states
            )
            v[curr_state][t] = max_prob + np.log(emission_probs[curr_state].get(obs_seq[t], 1e-6))
            backpointer[curr_state][t] = prev_state

    # Termination
    max_final_state = max(states, key=lambda state: v[state][-1])
    max_final_prob = v[max_final_state][-1]

    # Traceback
    best_path = [max_final_state]
    for t in range(n-1, 0, -1):
        best_path.insert(0, backpointer[best_path[0]][t])
    
    print(best_path)
    print(max_final_prob)

    return best_path, max_final_prob

# def viterbi(obs_seq, states, pi, A, E):
#     """
#     Viterbi algorithm for finding the most likely sequence of states.
    
#     Args:
#         obs_seq: The sequence of observations (indices corresponding to observed symbols).
#         states: List of hidden states.
#         pi: Initial probabilities (array of size [num_states]).
#         A: Transition probabilities (matrix of size [num_states x num_states]).
#         E: Emission probabilities (matrix of size [num_states x num_observations]).
    
#     Returns:
#         best_path: The most likely sequence of hidden states.
#         max_final_prob: Log probability of the best path.
#     """
#     n_states = len(states)
#     n = len(obs_seq)
    
#     v = np.full((n_states, n), -np.inf)  # Log probabilities (v[state, time])
#     backpointer = np.zeros((n_states, n), dtype=int)  # Backtracking table

#     # Initialization
#     for state in range(n_states):
#         v[state, 0] = np.log(pi[state]) + np.log(E[state, obs_seq[0]])

#     # Recursion
#     for t in range(1, n):
#         for curr_state in range(n_states):
#             # Find the max log probability for reaching `curr_state` at time `t`
#             max_prob, prev_state = max(
#                 (v[prev_state, t-1] + np.log(A[prev_state, curr_state]), prev_state)
#                 for prev_state in range(n_states)
#             )
#             v[curr_state, t] = max_prob + np.log(E[curr_state, obs_seq[t]])
#             backpointer[curr_state, t] = prev_state

#     # Termination
#     max_final_state = np.argmax(v[:, -1])
#     max_final_prob = v[max_final_state, -1]

#     # Traceback
#     best_path = [max_final_state]
#     for t in range(n-1, 0, -1):
#         best_path.insert(0, backpointer[best_path[0], t])

#     # Convert state indices to state labels
#     best_path = [states[state] for state in best_path]

#     return best_path, max_final_prob





#Model Evaluation

In [32]:
def calculate_accuracy(true_seq, pred_seq):
    assert len(true_seq) == len(pred_seq), "True and predicted sequences must have the same length."
    matches = sum(1 for true, pred in zip(true_seq, pred_seq) if true == pred)
    accuracy = matches / len(true_seq) * 100
    return accuracy

def calculate_error_percentage(true_seq, pred_seq):
    assert len(true_seq) == len(pred_seq), "True and predicted sequences must have the same length."
    mismatches = sum(1 for true, pred in zip(true_seq, pred_seq) if true != pred)
    error_percentage = mismatches / len(true_seq) * 100
    return error_percentage


states = ['H', 'E', 'C']
total_accuracy = 0
total_error = 0
n_sequences = len(hmm_data[:10]) 
initial_probs = compute_initial_probs(hmm_data)
emission_probs = compute_emission_probs(hmm_data)
transition_probs = compute_transition_probs(hmm_data)

for obs_seq, true_hid_seq in hmm_data[:10]:
    pred_hid_seq, prob = viterbi(obs_seq, states , initial_probs, emission_probs, transition_probs)
    accuracy = calculate_accuracy(true_hid_seq, pred_hid_seq)
    error_percentage = calculate_error_percentage(true_hid_seq, pred_hid_seq)

    print(f"True: {true_hid_seq}")
    print(f"Pred: {''.join(pred_hid_seq)}")
    print(f"Log Probability: {prob}")
    print(f"Accuracy: {accuracy:.2f}%")
    print(f"Error Percentage: {error_percentage:.2f}%\n")

    total_accuracy += accuracy
    total_error += error_percentage

average_accuracy = total_accuracy / n_sequences
average_error = total_error / n_sequences
print(f"Average Accuracy: {average_accuracy:.2f}%")
print(f"Average Error Percentage: {average_error:.2f}%")

state:  H
state:  E
state:  C
['C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H']
-571.9075313998836
True: CCCCCCCHHHCCCCCECHH